In [1]:
import tweepy 
import mysql.connector
from mysql.connector import Error
import json
from dateutil import parser
import time
import numpy as np

In [2]:
consumer_key = '******'
consumer_secret = '******'
access_token = '******'
access_token_secret = '******'

In [3]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [14]:
def connect(created_at, tweet, country, lon, lat, lon_real, lat_real, lang):
    try:
        con = mysql.connector.connect(host = 'localhost', port = 3306,
        database='twitter_stream', user='root', password = 'root', charset = 'utf8mb4')
        if con.is_connected():
            print('数据库连接成功')
            cursor = con.cursor()
            # twitter, golf
            query = "INSERT INTO tweets (created_at, tweet, country, lon, lat, lon_real, lat_real, lang) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
            cursor.execute(query, (created_at, tweet, country, lon, lat, lon_real, lat_real, lang))
            con.commit()


    except Error as e:
        print(e)

    cursor.close()
    con.close()

    return



In [23]:
#override tweepy.StreamListener to add logic to on_status
class MyStreamListener(tweepy.StreamListener):
    counts = 0
    
    def on_connect(self):
        print("Successfully connected to the Twitter API")

    def on_error(self, status):

        print(status)
        # returning false disconnects the stream
        return False

    #This method reads in tweet data as Json and extracts the data we want.

    def on_data(self,data):
        try:
            raw_data = json.loads(data)
            #print(raw_data)
            if raw_data['place'] is not None:
                self.counts += 1
                #print(raw_data)
                created_at = parser.parse(raw_data['created_at'])
                if 'extended_tweet' in raw_data:
                    tweet = raw_data['extended_tweet']['full_text']
                else:
                    tweet = raw_data['text']
                lang = raw_data['lang']
                country = raw_data['place']['country']
                coordinates = raw_data['place']['bounding_box']['coordinates']
                coor1 = coordinates[0][0]
                coor2 = coordinates[0][1]
                coor3 = coordinates[0][2]
                coor4 = coordinates[0][3]
                coor = np.array([coor1,coor2,coor3,coor4])
                mean_coor = coor.mean(axis=0)
                #print(mean_coor)
                lon = mean_coor[0]
                lat = mean_coor[1]
                if raw_data['coordinates'] is not None:
                    real_coord = raw_data['coordinates']
                    lon_real, lat_real = real_coord[0], real_coord[1]
                else:
                    lon_real, lat_real = 0.0, 0.0
                #insert data just collected into MySQL database
                connect(created_at, tweet, country, lon, lat, lon_real, lat_real, lang)
                print("Tweet colleted at: {}, {} fetched so far".format(str(created_at), str(self.counts)))
        except Error as e:
            print(e)


In [24]:
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener)
myStream.filter(track = ['covid','covid-19','covid19','pandemic'])

Successfully connected to the Twitter API
数据库连接成功
Tweet colleted at: 2021-05-17 09:50:54+00:00, 1 fetched so far
数据库连接成功
Tweet colleted at: 2021-05-17 09:50:56+00:00, 2 fetched so far
数据库连接成功
Tweet colleted at: 2021-05-17 09:51:01+00:00, 3 fetched so far
数据库连接成功
Tweet colleted at: 2021-05-17 09:51:17+00:00, 4 fetched so far
数据库连接成功
Tweet colleted at: 2021-05-17 09:51:19+00:00, 5 fetched so far
数据库连接成功
Tweet colleted at: 2021-05-17 09:51:21+00:00, 6 fetched so far
数据库连接成功
1062 (23000): Duplicate entry '2021-05-17 09:51:21' for key 'PRIMARY'
Tweet colleted at: 2021-05-17 09:51:21+00:00, 7 fetched so far
数据库连接成功
Tweet colleted at: 2021-05-17 09:51:36+00:00, 8 fetched so far
数据库连接成功
Tweet colleted at: 2021-05-17 09:51:39+00:00, 9 fetched so far
数据库连接成功
1062 (23000): Duplicate entry '2021-05-17 09:51:39' for key 'PRIMARY'
Tweet colleted at: 2021-05-17 09:51:39+00:00, 10 fetched so far
数据库连接成功
Tweet colleted at: 2021-05-17 09:51:41+00:00, 11 fetched so far
数据库连接成功
Tweet colleted at: 2021-05-

KeyboardInterrupt: 